In [5]:
import pyvista as pv
import numpy as np
%matplotlib qt

data = np.random.randn(300).reshape(-1, 3)
data[:,1] *= 3
data[:,0] *= 3

p = pv.Plotter(notebook=False)
p.add_mesh(pv.PolyData(data))
p.show()

[(-6.342894798417023, 41.133299612291275, -9.440514507723034),
 (0.3723180562346382, -0.22883897707778367, 0.010067278106367095),
 (-0.18935181140302698, 0.189395853911361, 0.9634703431032938)]

#### Point picking

In [24]:
hash(pv.PolyData(np.ones((3,3)))) == hash(pv.PolyData(np.ones((3,3))))

True

In [ ]:
def callback(polydata, pid):
    print(pid)

p.enable_point_picking(callback, use_mesh=True) 

In [1]:
# These two lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../..")

# Importing GemPy
import gempy as gp

# Embedding matplotlib figures in the notebooks
%matplotlib inline

# Importing auxiliary libraries
import numpy as np
import matplotlib.pyplot as plt
sys.path = list(np.insert(sys.path, 0, "../../../pyvista"))


#sys.path("../../../pyvista")
import pyvista as pv

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
geo_model = gp.load_model(
    'Tutorial_ch1-9b_Fault_relations', 
    path= '../data/gempy_models', 
    recompile=True
)

C:\Users\r02as18\miniconda3\envs\topology\lib\site-packages\theano\gpuarray\dnn.py:184: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn("Your cuDNN version is more recent than "


Active grids: ['regular']
Compiling theano function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  2
Compilation Done!


In [7]:
gp.compute_model(geo_model, compute_mesh=True)
# gp.plot.plot_section(geo_model, show_data=True)


Lithology ids 
  [7. 7. 7. ... 3. 3. 3.] 

In [3]:
import pyvista as pv
from PyQt5 import Qt
import sys
import numpy as np
from nptyping import Array


class MainWindow(Qt.QMainWindow):
    def __init__(self, geo_model, parent=None, show=True):
        Qt.QMainWindow.__init__(self, parent)
        self.geo_model = geo_model
        
        self.colors = self.geo_model.surfaces.df.color.values
        
        # create the frame
        self.frame = Qt.QFrame()
        vlayout = Qt.QVBoxLayout()

        # add the pyvista interactor object
        self.vtk_widget = pv.QtInteractor(self.frame)
        vlayout.addWidget(self.vtk_widget)

        self.frame.setLayout(vlayout)
        self.setCentralWidget(self.frame)

        # menu
        mainMenu = self.menuBar()
        fileMenu = mainMenu.addMenu('File')
        exitButton = Qt.QAction('Exit', self)
        exitButton.setShortcut('Ctrl+Q')
        exitButton.triggered.connect(self.close)
        fileMenu.addAction(exitButton)

        # allow adding a sphere
        meshMenu = mainMenu.addMenu('Plot')
        self.add_surfacepoints_action = Qt.QAction('Add Surface Points', self)
        self.add_surfacepoints_action.triggered.connect(self.add_surface_points)
        meshMenu.addAction(self.add_surfacepoints_action)
        
        self.add_surfaces_action = Qt.QAction('Add Surfaces', self)
        self.add_surfaces_action.triggered.connect(self.add_surfaces)
        meshMenu.addAction(self.add_surfaces_action)
        
        self.vtk_widget.enable_point_picking() 

        if show:
            self.show()
        
    def add_surface_points(self):
        points = pv.PolyData(
            self.geo_model.surface_points.df[["X", "Y", "Z"]].values
        )
        self.vtk_widget.add_mesh(points)
        
    def add_surfaces(self):
        vertices = self.geo_model.solutions.vertices
        simplices = self.geo_model.solutions.edges
        
        for i, (ver, sim) in enumerate(zip(self.geo_model.solutions.vertices, 
                            self.geo_model.solutions.edges)):
            sim = self._simplices_to_pv_tri_simplices(sim)
            
            self.vtk_widget.add_mesh(pv.PolyData(ver, sim), color=self.colors[i])
            
    def _simplices_to_pv_tri_simplices(self, sim:Array[int, ..., 3]) -> Array[int, ..., 4]:
        """Convert triangle simplices (n, 3) to pyvista-compatible
        simplices (n, 4)."""
        n_edges = np.ones(sim.shape[0]) * 3
        return np.append(n_edges[:, None], sim, axis=1)


In [6]:
from PyQt5 import QtCore
from PyQt5.QtWidgets import QWidget

In [ ]:
%gui qt

app = QtCore.QCoreApplication.instance()
if app is None:
    app = Qt.QApplication(sys.argv)
    
w = QWidget()
w.setWindowTitle('Simple')
w.show()
window = MainWindow(geo_model)
# sys.exit(app.exec_())
app.exec_()

In [3]:
from gempy.plot import vista
from importlib import reload
reload(vista)

<module 'gempy.plot.vista' from '../..\\gempy\\plot\\vista.py'>

In [4]:
import pyvista as pv

In [1]:
from create_geomodel_gullfaks import create_geomodel

ModuleNotFoundError: No module named 'create_geomodel_gullfaks'

In [5]:
gv = vista.Vista(geo_model, notebook=False, plotter_type="background")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [40]:
gv.plot_surfaces()

,val
7,PolyData (0x23e0d48e468)\n N Cells:\t7154\n ...
5,PolyData (0x23e0d48e6a8)\n N Cells:\t12152\n ...
6,PolyData (0x23e0d48eb28)\n N Cells:\t5824\n ...
0,PolyData (0x23e0d48ea08)\n N Cells:\t5684\n ...
3,PolyData (0x23e0d3b7a68)\n N Cells:\t5770\n ...
1,PolyData (0x23e0d3b7a08)\n N Cells:\t5684\n ...


In [115]:
class Vista2:
    def __init__(self, geo_model, **kwargs):
        self.geo_model = geo_model
        self.p = pv.BackgroundPlotter(**kwargs)
        self.entities = []
        
    def plot_surfaces(self):
        for idx, val in self.geo_model.surfaces.df.dropna().iterrows():
            if idx in self.entities:
                continue
                
            surf = pv.PolyData(
                val.vertices, 
                np.insert(val['edges'], 0, 3, axis=1).ravel()
            )
            self.p.add_mesh(surf, color=val['color'])
            self.entities.append(idx)
            

gv2 = Vista2(geo_model)

In [116]:
gv2.plot_surfaces()